## General Exploration
The goal of this phase is to get a general feel for the data and identify possible cleaning, feature engineering, and feature selection tasks.

In [1]:
import numpy as np
import pandas as pd
from settings import db_password, db_server, db_user

import src.features.data_cleaning as data_clean
import src.features.data_exploration as data_exp

In [2]:
# these 'magic' functions allow you to adjust your code in an editor (e.g. Pycharm) and directly apply the changes in this notebook
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [3]:
df = data_exp.return_dwh_table(
    db_user, db_password, db_server, sql_query="SELECT * FROM eb_toon_churn_cltv"
)

In [4]:
df.head(n=10)

,id_parent_klant,toon_churn,code_org_eigenaar_dwh,model_peildatum,ele_klant_ind,ele_contr_freq,ele_lpt_1jr_ind,ele_lpt_2jr_ind,ele_lpt_3jr_ind,ele_lpt_4jr_ind,...,app_sessie_last_dat,app_mobile_sessies_freq,app_saldo_overview_ind,app_voorschot_check_ind,app_transactie_aanvr_ind,app_voorschot_aanpassing_ind,app_energieprofiel_toevoeg_ind,app_bepaal_betaaldag_ind,app_mandaat_ja_ind,app_mandaat_nee_ind
0,MI-46524513,0,ENECORET,2020-01-01,1,1,0,0,0,1,...,NaT,0,0,0,0,0,0,0,0,0
1,MI-46523434,0,ENECORET,2020-01-01,1,1,0,0,0,1,...,NaT,0,0,0,0,0,0,0,0,0
2,MI-46520280,0,ENECORET,2020-01-01,1,1,0,0,0,1,...,NaT,0,0,0,0,0,0,0,0,0
3,MI-46556716,1,ENECORET,2020-01-01,0,0,0,0,0,0,...,NaT,0,0,0,0,0,0,0,0,0
4,MI-46556286,0,ENECORET,2020-01-01,1,1,0,0,0,1,...,NaT,0,0,0,0,0,0,0,0,0
5,MI-45046840,1,ENECORET,2020-01-01,0,0,0,0,0,0,...,2019-01-05,2,0,0,0,0,0,0,0,0
6,MI-46038838,1,ENECORET,2020-01-01,0,0,0,0,0,0,...,2019-05-17,41,1,0,1,0,0,0,0,0
7,MI-45242170,1,ENECORET,2020-01-01,0,0,0,0,0,0,...,NaT,0,0,0,0,0,0,0,0,0
8,MI-685425,0,ENECORET,2020-01-01,1,1,0,0,0,0,...,NaT,0,0,0,0,0,0,0,0,0
9,MI-45154073,1,ENECORET,2020-01-01,0,0,0,0,0,0,...,NaT,0,0,0,0,0,0,0,0,0


### notes describe_df
- A Toon churner is likely to not longer be a customer of Eneco, thereby creating a lot of missing values for regularly
updated data fiels
- Which data is relevant up to the point of churning? 
- Can we retrieve these snapshots?

In [5]:
data_exp.describe_df(df, dependent_variable="toon_churn")

This dataframe has 57246 rows and 416 columns.

The dependent variable consists of 50.0% of ones.

The variables have the following data types:
int64             378
float64            19
datetime64[ns]     12
object              7
dtype: int64

The postfixes are distributed as follows:
Counter({'ind': 333, 'freq': 42, 'dat': 13, 'cat': 9, 'num': 8, 'rec': 7, 'klant': 1, 'churn': 1, 'dwh': 1, 'peildatum': 1})

The following variables have missing values:
                             column  perc_missings
325         ftr_dagen_sinds_lst_rec       1.000000
324       contact_ftr_lst_datum_dat       1.000000
322      klacht_dagen_sinds_lst_rec       0.997939
321    contact_klacht_lst_datum_dat       0.997939
381    web_tablet_laatst_actief_dat       0.942965
379  web_pc_datum_laatst_actief_dat       0.762429
406             app_sessie_last_dat       0.691472
383    web_mobile_laatst_actief_dat       0.660378
389      web_mijn_laatst_actief_dat       0.568861
377     web_datum_laatst_actief

### Set aside a test set 
It may feel counter intuitive to set aside a significant part of the data at the beginning but to get the most reliable generalization error this step is *absolutely necessary*. 
- It is common practice to set aside 20% as a test set but with huge amounts of data this percentage should be lower 
- Note that all the transformations on the train set must also be applied on the test set
- The train_set is also exported since the connection to the DWH is slow and loading the dataset from disk is faster

In [ ]:
train_set = data_exp.set_aside_test_set(df, test_size=0.2, random_state=1337)

In [ ]:
del df  # no longer needed

### notes odds_ind 
- pay attention to the sum(y) variables, when (very) low the absolute and relative differences can be misleading
- btwz_onbekend_ind -> obvious since a Toon customer will most likely leave, thereby stops paying
- web_url_verhuizen_lst_jr_ind -> interesting & expected 
- incgrp_medium_risk_ind -> why? 

In [ ]:
df_odds_ind = data_exp.odds_ind(train_set, dependent_variable="toon_churn")

In [ ]:
df_odds_ind.iloc[:25]

### notes freq insight
- cont_kcm_admin_billing_freq > 0, chance on churn increases
- remove variables from dataframe when abs < 0.05 
- convert other low variance variables to indicators or combine (e.g., all sustainability variables)

In [ ]:
data_exp.freq_insight(train_set)

In [ ]:
pd.crosstab(train_set["cont_kcm_admin_billing_freq"], train_set["toon_churn"]).assign(
    perc_churn=lambda x: x[1] / np.sum(x, axis=1)
)[:10]

### notes dates
- Not all variables with a '_dat' postfix are actual datetimes
- OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 9999-12-31 00:00:00
- If an einddatum is known then chances are that customers will stop using Toon, is this too obvious? In case no einddatum
is known still 25% churns on Toon (in this dataset)
- contact_ftr_lst_datum_dat can be removed (standard via nzv?)
- ltst_sale_werving_dat and ltst_sale_dat can be converted to days since today:
```python
today = datetime.date.today()
(today - df['ltst_sale_werving_dat'].dt.date).dt.days
```
but how to handle missings?
- The same goes for web & app data

In [ ]:
dates_list = data_exp.postfix_to_column(
    train_set, postfix="dat", dependent_variable=None
)

In [ ]:
train_set[dates_list].dtypes

In [ ]:
train_set[dates_list].head()

In [ ]:
train_set["max_eind_levering_ind"] = np.where(
    train_set["klant_max_eind_levering_dat"].astype("str") == "9999-12-31 00:00:00",
    1,
    0,
)

In [ ]:
pd.crosstab(train_set["max_eind_levering_ind"], train_set["toon_churn"]).assign(
    perc_churn=lambda x: x[1] / np.sum(x, axis=1)
)

In [ ]:
import datetime

In [ ]:
today = datetime.date.today()
print(today)
print((today - df["ltst_sale_werving_dat"].dt.date).dt.days)

### Notes categorical variables
- fix cat_visu function: energie_label should get it's own distribution instead of auto remove
- energie_label_incl_vrlopig_cat indicator for label = A? -> significant higher churn percentage (add unknown and fillna?)

In [ ]:
data_exp.cat_visu(train_set, dependent_variable="toon_churn")

In [ ]:
pd.crosstab(
    train_set["energie_label_incl_vrlopig_cat"], train_set["toon_churn"]
).assign(perc_churn=lambda x: x[1] / np.sum(x, axis=1))

### Notes numerical variables
- log_trans_list = ['ele_jvb_geschat_tot_num', 'woon_avg_vloeropvlak_num']
- Indicator functions for incasso_* variables * woon_avg_vloeropvlak_num
- qcut functions for ele_jvb_*

In [ ]:
data_exp.num_visu(train_set)

### Notes rec variables
- How to go about missings? Imputation make no sense in cae of complaints .. indicators? 
- Are they already present?
- klacht_dagen_sinds_lst_rec & ftr_dagen_sinds_lst_rec can be removed due to lack of information

In [ ]:
rec_list = data_exp.postfix_to_column(train_set, postfix="rec", dependent_variable=None)

In [ ]:
train_set[rec_list].head()

In [ ]:
train_set[rec_list].info()

### Notes correlation
- Toon_abo_klant_ind should be removed for obvious reasons
- A lot of other variables are extremely highly correlated and should be removed due to redundancy 
- Discuss with the team for future training datasets

```python
use list(df_multi_corr['drop_suggestion']) to produce the drop_list
```

In [ ]:
df_multi_corr = data_exp.multicollinearity(
    train_set, cut_off=0.9, dependent_variable="toon_churn"
)

In [ ]:
with pd.option_context("display.max_rows", 100):
    display(df_multi_corr.sort_values(by="corr", ascending=False))

### Notes PCA
- Not surpisingly the results indicates that a lot of the variance can be captured in just 3 dimensions
- Try this modeling route -> saves time :)

In [ ]:
df_pca_obj = data_exp.pca_visu(train_set, dependent_variable="toon_churn")

In [ ]:
cumulative_variance = pd.Series(np.cumsum(df_pca_obj.explained_variance_ratio_))
cumulative_variance[cumulative_variance < 0.98]